In [1]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-y20mgo4g
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-y20mgo4g


In [26]:
pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 413 kB/s eta 0:46:16tcmalloc: large alloc 1147494400 bytes == 0x5616f8f70000 @  0x7f05a72e1615 0x5616bfce13bc 0x5616bfdc218a 0x5616bfce41cd 0x5616bfdd6b3d 0x5616bfd58458 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd582c0 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd54cd4 0x5616bfdd7986 0x5616bfd54350 0x5616bfdd7986 0x5616bfd54350 0x5616bfdd7986 0x5616bfd54350 0x5616bfce5f19 0x5616bfd29a79 0x5616bfce4b32 0x5616bfd581dd 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd54cd4 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd53eae 0x5616bfce59da 0x5616bfd54108 0x5616bfd5302f
     |█████████████████               | 1055.7 MB 432 kB/s eta 0:35:45tcmalloc: large alloc 1434370048 bytes == 0x56173d5c6000 @  0x7f05a72e1615 0x5616bfce13bc 0x5616bfdc218a 0x5616bfce41cd 0x5616bfdd6b3d 0x5616bfd58458 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd582c0 0x5616bfd5302f 0x5616bfce5aba 0x5616bfd54cd4 0x5616bfdd7986 0x5616bfd543

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

import os

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
device = torch.device("cuda")
# device = torch.device("cpu")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
dataset_train = []
dataset_test = []

root = "/content/drive/MyDrive/news_class9x1400/"
list = os.listdir(root)
categories = {category:i for i,category in enumerate(list)}

for cat in categories:
    files = os.listdir(root + cat)
    for i,f in enumerate(files):
        fname = root + cat + "/" + f
        file = open(fname, "r", encoding="utf-8")
        strings = file.read()
        if i<1120:
            dataset_train.append([strings, categories[cat]])
        else:
            dataset_test.append([strings,categories[cat]])
        file.close()

print(len(dataset_train), len(dataset_test))
    

10080 2520


In [10]:
categories

{'ITscience': 0,
 'culture': 5,
 'economy': 1,
 'entertainment': 6,
 'health': 2,
 'life': 7,
 'politic': 8,
 'social': 3,
 'sport': 4}

In [11]:
dataset_train[0]

['기타 신동도 이 대통령도 유튜브로 돈 벌 수 있다\n한국 개인 사용자들이 전 세계를 상대로 수익을 얻을 기회다.\n온라인 동영상 사이트 유튜브는 6일 오전 10시 30분 서울 역삼동 구글코리아 본사에서 유튜브 3.0을 소개하는 기자간담회를 열었다.\n유튜브는 하루 조회수 30억 건에 이르는 세계 최대 온라인 동영상 사이트로 한국을 포함해 전세계 32개국에서 서비스하고 있다.\n구글 검색, 지메일 등 다른 서비스들이 네이버, 다음과 같은 토종 포털에 밀려 한국 시장에서 고전하고 있는 와중에도 유튜브는 국내 동영상 사이트 1위를 고수해 왔다.\n유튜브는 이날 2008년 1월 한국 서비스 당시 단순히 사용자들이 동영상을 올리고 나누는 개념이었던 1.0세대, SM, JYP, YG 등 국내 파트너사들과 수익을 배분하는 2.0 세대에 이어 개인 사용자들과도 광고 수익을 나누는 3.0 세대 개념을 발표했다.\n일본 도쿄에서 화상으로 연결된 아담 스미스 유튜브 아시아태평양지역 총괄은 유튜브는 이제 스마트폰, 태블릿, TV 등을 통해 언제, 어디서, 어떤 기기로든 시청할 수 있다면서 창의적 사용자들이 동영상에 광고를 붙여 수익화할 수 있게 유튜브가 도와줄 것이라고 밝혔다.\n스미스 총괄은 기타 연주 동영상을 유튜브에 올려 3억 회 이상 조회수를 기록한 14살 기타 신동 정성하군 사례를 들며 파트너를 일반 사용자까지 확대하겠다고 밝혔다.\n그동안 동영상 저작권 침해 문제로 콘텐츠 사업자들과 크고 작은 마찰을 빚었던 유튜브는 저작권 보호 솔루션인 콘텐츠 ID 기술을 도입해 세계 미디어 기업과 파트너 관계를 맺고 있다.\n콘텐츠 ID는 파트너사에서 동영상을 올린 뒤 일반 사용자들이 같은 동영상을 올릴 경우 이를 가려내 파트너 스스로 저작권을 관리할 수 있게 하는 기술이다.\n스미스는 콘텐츠ID 덕분에 파트너사 저작권을 보호하고 수익 창출이 가능해 세계 굴지 미디어 기업과 파트너를 맺을 수 있다, 아시아에선 한국 K팝과 한류가 성공 사례로 꼽힌다면서 SM, YG, JYP 등 한국

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
l1 = [len(i[0]) for i in dataset_train]
l2 = [len(i[0]) for i in dataset_test]
max(max(l1),max(l2))

4911

In [15]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        #valid_length 까지만 1, 나머지는 0으로 mask를 생성
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        # return_dict=False 추가
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        #pooler는 CLS토큰에 대한 임베딩의 결과, 단순히 첫번째 토큰에 대한 결과가 아니라 문맥을 알아내기 위한 추가 layer를 통과한다.(아마도?)
        #임베딩의 결과에 linear layer를 통과시켜서 classification을 진행한다.
        # print(pooler.shape) batchsize * 768

        #_는 64개(max_length)의 모든 토큰에 대한 임베딩의 결과
        #단어 임베딩을 알고 싶을 때 사용할 수 있다.
        #_의 첫번째 값과 pooler와는 다른 값을 가지고 있는데 pooler는 _의 첫번째 값의 추가적으로 한번더 과정을 거친다.
        #print(_.shape) batchsize * max_len * 768

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) #batchsize * num_classes

In [19]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [20]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [22]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [23]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        # if batch_id % log_interval == 0:
        #     print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/158 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.4285996835443038


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.7927083333333333


  0%|          | 0/158 [00:00<?, ?it/s]

epoch 10 train acc 0.9920886075949367


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 10 test acc 0.7954427083333333


In [26]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

In [ ]:
# cate = ["정치","경제","사회", "생활/문화","세계","기술/IT", "연예", "스포츠"]
# tmp = ["아이패드 프로에 m1칩 탑재"]
# transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
# tokenized = transform(tmp)

# result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
# idx = result.argmax().cpu().item()
# print("뉴스의 카테고리는:", cate[idx])
# print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))


In [27]:
torch.save(model.state_dict(), "news.pt")
modelload = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
modelload.load_state_dict(torch.load("news.pt", device))
# modelload.eval()

<All keys matched successfully>

In [34]:
def testModel(model, seq):
    cate = ['ITscience', 'economy', 'health', 'social', 'sport', 'culture', 'entertainment', 'life', 'politic']
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    modelload.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("뉴스의 카테고리는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [35]:
testModel(model, "대통령 투표는 3월 9일입니다.")

뉴스의 카테고리는: politic
신뢰도는: 97.22%
